# Traductions 

Nous allons voir qu'avec les LSTM, on peut faire des choses assez puissantes comme : *des traducteurs* ! Voyons comment nous pouvons créer un traducteur français > anglais avec TensorFlow 

## Description du projet 

Ce projet n'est pas des plus faciles. C'est pourquoi nous allons vous éclairer sur les grandes étapes à suivre. 

### Import des données 

Tout d'abord, vous aurez un fichier `.txt` contenant une phrase avec sa traduction séparée par une tabulation (`\t`). Vous devrez donc importer ces données et les lire via `pandas` ou `numpy`. 

Vos données se trouvent sur ce lien : https://go.aws/38ECHUB

### Preprocessing 

Tout l'objectif de votre preprocessing est d'arriver à exprimer votre phrase d'entrée (française) en une séquence d'indices.

i.e :

* je suis malade ---> `[123, 21, 34, 0, 0, 0, 0]`

Ce qui donne une *shape* -> `(batch_size, max_len_of_a_french_sentence)`

Les indices correspondent à un numéro que vous devrez attribuer pour chaque token de mots. 

Les zéros correspondent à ce qu'on appelle des [*padded_sequences*](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences) qui permettent le fait que tous les séquences de mots aient la même longueur (obligatoire pour votre algorithme). 

La transformation de votre phrase cible ne sera pas exactement la même que celle de votre phrase d'entrée. En plus de toutes les étapes que vous aurez effectué pour les phrases d'entrées, vous devrez en plus *catégoriser* votre phrase cible. Autrement dit, un exemple de tenseur ressemblera à : 

* I am sick ---> 

```
[
  [1, 0, 0, ..., 0, 0],
  [0, 0, 0, ..., 1, 0], 
  ...
  [0, 1, 0, ..., 0, 0]

]
```
Ce qui donne une *shape* -> `(batch_size, max_len_of_an_english_sentence, num_of_classes)`

Pour aider à effectuer cela, vous pourrez utiliser : 

* `Pandas` ou `Numpy` pour la lecture du fichier text
* `Spacy` pour la Tokenisation 
* `Tensorflow` pour le [padded_sequence](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences) & la [catégorisation](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical?hl=en) 

### Modélisation 

Une fois que vous aurez cela, vous pourrez passer à la modélisation. Pour créer votre modèle, vous aurez besoin : 

* D'une couche d'[Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding?hl=en)
* De couches [LSTM](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM?hl=en) & [Bidirectionnelles](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional?hl=en)
* D'une couche [RepeatVector](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RepeatVector?hl=en)
* D'une couche [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense?hl=en) & [TimeDistributed](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TimeDistributed?hl=en) en sortie

L'objectif étant d'avoir en entrée, un tenseur de dimension `(batch_size, max_len_of_a_french_sentence)` et en sortie un tenseur de dimension `(batch_size, max_len_of_an_english_sentence, num_of_classes)` où évidemment `max_len_of_a_french_sentence` $\neq $ `max_len_of_an_english_sentence`


### Conseils 

Ne prenez pas l'entièreté du dataset au départ pour vos expérimentations, prenez simplement 5000 voire même 3000 phrases. Cela vous permettra d'itérer plus vite et d'éviter des bugs liés simplement à votre besoin en puissance de calcul. 

Good Luck !



In [4]:
!pip install --upgrade tensorflow 

In [5]:
# Import des librairies nécessaires
import pandas as pd
import numpy as np 
import tensorflow_datasets as tfds
import tensorflow as tf 
tf.__version__

'2.13.0'

## Import des données 

In [6]:
# Fonction de chargement du document txt
def load_doc(url):
  df = pd.read_csv(url, delimiter="\t", header=None)
  return df

In [7]:
# Chargement du document txt
doc = load_doc("https://go.aws/38ECHUB")
doc.head()

,0,1
0,Go.,Va !
1,Hi.,Salut !
2,Run!,Cours !
3,Run!,Courez !
4,Wow!,Ça alors !


In [8]:
# Prenons simplement un sample de 5000 phrases pour éviter des lenteurs 
doc = doc.sample(5000)

In [9]:
# Chargement des langages français et anglais de spacy 
!python -m spacy download fr_core_news_sm
!python -m spacy download en_core_web_sm

     -------------------------------------- 16.3/16.3 MB 469.0 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
     -------------------------------------- 12.8/12.8 MB 480.0 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [10]:
# Import de chacun des langages
import fr_core_news_sm
import en_core_web_sm
nlp_fr = fr_core_news_sm.load()
nlp_en = en_core_web_sm.load()

In [11]:
# Chargement du corpus entier de phrases françaises et anglaises
fr_corpus = " ".join(doc.iloc[:, 1].to_list())
en_corpus = " ".join(doc.iloc[:, 0].to_list())

In [12]:
fr_corpus

'Afin d\'être à l\'heure, je courus. Contactez-moi dès que vous arrivez ici. Tu ne devrais pas attendre ici. Pouvez-vous me dire ce que Tom a fait\u202f? Quelqu\'un a coupé la corde. Il est trop tôt pour que tu viennes. Je me demande ce qui les fait rire. Elle doit avoir dit un mensonge. J\'ignore pourquoi vous êtes tous tellement en colère. On s\'est tellement amusé ensemble. Il y avait une grosse pluie hier. Laquelle de ces raquettes est la tienne ? J\'ai besoin d\'un ordinateur portable. Laisse-moi voir ce que tu as dans la main ! Ce genre de personne ne demande presque jamais d\'augmentation. Ne sors pas sans dire au revoir. Il a d\'excellents réflexes. Nous le ferons ensemble. Il y a encore beaucoup de travail à faire. Rencontrer des garçons est difficile. Je pense que Tom a fait cela intentionnellement. Je suis né en octobre. Il se tourna vers ses amis pour obtenir de l\'aide. Elles disposent de suffisamment d\'argent. Passes-tu du bon temps ? Je n\'aime aucune d\'entre elles. Tu

In [36]:
# Chargement des deux corpus dans spacy 
#A ETE CHANGEE
#%%time
#import time
#nlp_fr.max_length = len(fr_corpus)
#nlp_en.max_length = len(en_corpus)

fr_doc = nlp_fr(fr_corpus)
en_doc = nlp_en(en_corpus)

In [37]:
#A ETE CHANGEE
#  Tokenisation de chacune des phrases via spacy 
#%%time
doc["fr_tokens"] = doc.iloc[:, 1].apply(lambda x: nlp_fr.tokenizer(x))
doc["en_tokens"] = doc.iloc[:, 0].apply(lambda x: nlp_en.tokenizer(x))

In [38]:
doc.tail()

,0,1,fr_tokens,en_tokens
84585,There was nothing left to do.,Il n'y avait plus rien à faire.,"(Il, n', y, avait, plus, rien, à, faire, .)","(There, was, nothing, left, to, do, .)"
44836,I need to go into town.,J'ai besoin d'aller en ville.,"(J', ai, besoin, d', aller, en, ville, .)","(I, need, to, go, into, town, .)"
150888,"I know you probably want to be alone, so I'll ...",Je sais que vous voulez probablement vous retr...,"(Je, sais, que, vous, voulez, probablement, vo...","(I, know, you, probably, want, to, be, alone, ..."
140935,Remove the chicken's giblets before cooking.,Ôtez les abats du poulet avant de le cuire.,"(Ôtez, les, abats, du, poulet, avant, de, le, ...","(Remove, the, chicken, 's, giblets, before, co..."
156949,"No matter how you look at it, the odds are sta...",Qu'importe la manière dont tu considères la ch...,"(Qu', importe, la, manière, dont, tu, considèr...","(No, matter, how, you, look, at, it, ,, the, o..."


In [39]:
# Création d'un set() qui va prendre tous les tokens unique de notre corpus de texte
en_tokens = [token.text for token in en_doc]
en_vocabulary_set= set(en_tokens) # retirer tous les doublons
en_vocab_size = len(en_vocabulary_set)
print(en_vocab_size)

3699


In [40]:
# Même chose pour le français 
fr_tokens = [token.text for token in fr_doc]
fr_vocabulary_set= set(fr_tokens) # retirer les doublons
fr_vocab_size = len(fr_vocabulary_set)
print(fr_vocab_size)

5075


In [41]:
# Création d'un id pour chacun des tokens
all_en_tokens = {}
for i,en_token in enumerate(en_vocabulary_set):
  all_en_tokens[en_token] = i+1 # On prend à i+1 pour laisser la valeur 0 pour la création des padded_sequences

all_fr_tokens = {}
for i, fr_token in enumerate(fr_vocabulary_set):
  all_fr_tokens[fr_token] = i+1

In [42]:
# Création de fonction qui vont créer un vecteur d'indices pour chacune des séquences de tokens
def en_tokens_to_index(tokens):
  indices = []
  for token in tokens:
    indices.append(all_en_tokens[token.text])
  
  return indices

def fr_tokens_to_index(tokens):
  indices = []
  for token in tokens:
    indices.append(all_fr_tokens[token.text])
  
  return indices

In [43]:
# Transformation des tokens en indices
doc["fr_indices"] = doc["fr_tokens"].apply(lambda x: fr_tokens_to_index(x))
doc["en_indices"] = doc["en_tokens"].apply(lambda x: en_tokens_to_index(x))

In [44]:
doc.tail()

,0,1,fr_tokens,en_tokens,fr_indices,en_indices
84585,There was nothing left to do.,Il n'y avait plus rien à faire.,"(Il, n', y, avait, plus, rien, à, faire, .)","(There, was, nothing, left, to, do, .)","[71, 3624, 1516, 1964, 611, 1524, 2791, 1881, ...","[2196, 2976, 3624, 1829, 3273, 1695, 2044]"
44836,I need to go into town.,J'ai besoin d'aller en ville.,"(J', ai, besoin, d', aller, en, ville, .)","(I, need, to, go, into, town, .)","[1061, 180, 2655, 2657, 1987, 3147, 2260, 3261]","[1385, 2705, 3273, 1094, 3013, 2918, 2044]"
150888,"I know you probably want to be alone, so I'll ...",Je sais que vous voulez probablement vous retr...,"(Je, sais, que, vous, voulez, probablement, vo...","(I, know, you, probably, want, to, be, alone, ...","[3295, 1316, 1233, 795, 4316, 4845, 795, 1924,...","[1385, 2876, 1200, 432, 1377, 3273, 591, 2182,..."
140935,Remove the chicken's giblets before cooking.,Ôtez les abats du poulet avant de le cuire.,"(Ôtez, les, abats, du, poulet, avant, de, le, ...","(Remove, the, chicken, 's, giblets, before, co...","[4456, 3774, 3125, 3911, 1127, 4056, 2362, 434...","[1740, 735, 2338, 500, 1623, 665, 1185, 2044]"
156949,"No matter how you look at it, the odds are sta...",Qu'importe la manière dont tu considères la ch...,"(Qu', importe, la, manière, dont, tu, considèr...","(No, matter, how, you, look, at, it, ,, the, o...","[562, 2622, 2313, 3638, 994, 2009, 4935, 2313,...","[3498, 3199, 3097, 1200, 3160, 980, 2162, 2368..."


In [45]:
# Création d'une fonction qui va compter la longueur maximum d'une phrase
def max_len(lines):
  return max(len(line) for line in lines)

In [46]:
# Application de la fonction sur les tokens français et anglais 
fr_max_len = max_len(doc['fr_indices'].to_list())
en_max_len = max_len(doc['en_indices'].to_list())

In [47]:
tf.keras.preprocessing.sequence.pad_sequences?

Signature:
tf.keras.preprocessing.sequence.pad_sequences(
    sequences,
    maxlen=None,
    dtype='int32',
    padding='pre',
    truncating='pre',
    value=0.0,
)
Docstring:
Pads sequences to the same length.

This function transforms a list (of length `num_samples`)
of sequences (lists of integers)
into a 2D Numpy array of shape `(num_samples, num_timesteps)`.
`num_timesteps` is either the `maxlen` argument if provided,
or the length of the longest sequence in the list.

Sequences that are shorter than `num_timesteps`
are padded with `value` until they are `num_timesteps` long.

Sequences longer than `num_timesteps` are truncated
so that they fit the desired length.

The position where padding or truncation happens is determined by
the arguments `padding` and `truncating`, respectively.
Pre-padding or removing values from the beginning of the sequence is the
default.

>>> sequence = [[1], [2, 3], [4, 5, 6]]
>>> tf.keras.utils.pad_sequences(sequence)
array([[0, 0, 1],
       [0, 2,

In [49]:
# Utilisation de Keras pour créer des séquences de tokens de la même longueur
#%%time
padded_fr_indices = tf.keras.preprocessing.sequence.pad_sequences(doc["fr_indices"], maxlen=fr_max_len, padding="post")
padded_en_indices = tf.keras.preprocessing.sequence.pad_sequences(doc["en_indices"], maxlen=en_max_len, padding="post")

In [50]:
# Visualisation de la shape d'un des tenseurs
padded_fr_indices.shape

(5000, 44)

In [51]:
# Application de la catégorisation de la variable cible 
binarized_en_indices = tf.keras.utils.to_categorical(padded_en_indices, num_classes=en_vocab_size+1)
binarized_en_indices.shape

(5000, 38, 3700)

In [52]:
tf.data.Dataset.from_tensor_slices?

Signature: tf.data.Dataset.from_tensor_slices(tensors, name=None)
Docstring:
Creates a `Dataset` whose elements are slices of the given tensors.

The given tensors are sliced along their first dimension. This operation
preserves the structure of the input tensors, removing the first dimension
of each tensor and using it as the dataset dimension. All input tensors
must have the same size in their first dimensions.

>>> # Slicing a 1D tensor produces scalar tensor elements.
>>> dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
>>> list(dataset.as_numpy_iterator())
[1, 2, 3]

>>> # Slicing a 2D tensor produces 1D tensor elements.
>>> dataset = tf.data.Dataset.from_tensor_slices([[1, 2], [3, 4]])
>>> list(dataset.as_numpy_iterator())
[array([1, 2], dtype=int32), array([3, 4], dtype=int32)]

>>> # Slicing a tuple of 1D tensors produces tuple elements containing
>>> # scalar tensors.
>>> dataset = tf.data.Dataset.from_tensor_slices(([1, 2], [3, 4], [5, 6]))
>>> list(dataset.as_numpy_it

In [53]:
# Création de tf.data.Dataset pour chacun des tenseurs français et anglais
fr_ds = tf.data.Dataset.from_tensor_slices(padded_fr_indices)
en_ds = tf.data.Dataset.from_tensor_slices(binarized_en_indices)

In [54]:
# Create a tensorflow dataset complet
tf_ds = tf.data.Dataset.zip((fr_ds, en_ds))

In [55]:
next(iter(tf_ds))

(<tf.Tensor: shape=(44,), dtype=int32, numpy=
 array([ 709, 2657,  579, 2791, 2991, 1539,  843, 2402, 3372, 3261,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0])>,
 <tf.Tensor: shape=(38, 3700), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)>)

In [56]:
# Shuffle & Batch
BATCH_SIZE = 32

tf_ds = tf_ds.shuffle(len(doc)).batch(BATCH_SIZE)

In [57]:
len(doc)/32*0.7

109.375

In [58]:
# Train Test Split
TAKE_SIZE = int(0.7*len(doc)/BATCH_SIZE)

train_data = tf_ds.take(TAKE_SIZE)
test_data = tf_ds.skip(TAKE_SIZE)

In [59]:
tf.keras.layers.Embedding?

Init signature:
tf.keras.layers.Embedding(
    input_dim,
    output_dim,
    embeddings_initializer='uniform',
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=None,
    sparse=False,
    **kwargs,
)
Docstring:     
Turns positive integers (indexes) into dense vectors of fixed size.

e.g. `[[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]`

This layer can only be used on positive integer inputs of a fixed range. The
`tf.keras.layers.TextVectorization`, `tf.keras.layers.StringLookup`,
and `tf.keras.layers.IntegerLookup` preprocessing layers can help prepare
inputs for an `Embedding` layer.

This layer accepts `tf.Tensor`, `tf.RaggedTensor` and `tf.SparseTensor`
input.

Example:

>>> model = tf.keras.Sequential()
>>> model.add(tf.keras.layers.Embedding(1000, 64, input_length=10))
>>> # The model will take as input an integer matrix of size (batch,
>>> # input_length), and the largest integer (i.e. word index) i

In [60]:
# Création du modèle 
model = tf.keras.Sequential([
                  # Couche d'Input Word Embedding           
                  tf.keras.layers.Embedding(fr_vocab_size+1, 64, mask_zero=True),

                  # Couche LSTM Bidirectionnelle
                  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                  
                  # Nouvelle couche LSTM Bidirectionnel
                  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),

                  # Repeat Vector
                  tf.keras.layers.RepeatVector(binarized_en_indices.shape[1]),

                  # Nouvelle couche LSTM
                  tf.keras.layers.LSTM(32, return_sequences=True),               

                  # Couche de sortie avec le nombre de neurones en sortie égale au nombre de classe avec fonction softmax
                  tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(en_vocab_size+1, activation="softmax"))
           
])

In [61]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          324864    
                                                                 
 bidirectional (Bidirection  (None, None, 128)         66048     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 repeat_vector (RepeatVecto  (None, 38, 128)           0         
 r)                                                              
                                                                 
 lstm_2 (LSTM)               (None, 38, 32)            20608     
                                                        

In [62]:
# Prédiction "aléatoire" pour tester notre modèle 
input_text, output_text = next(iter(train_data))
print(input_text.numpy().shape)
print(model.predict(input_text).shape)
print(output_text.numpy().shape)

(32, 44)
1/1 [==============================] - 9s 9s/step
(32, 38, 3700)
(32, 38, 3700)


In [63]:
# Créons un learning rate schedule pour décroitre le learning rate à mesure que nous entrainons le modèle 
initial_learning_rate = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1090,
    decay_rate=0.96,
    staircase=True)

# Utilisation d'un compileur simple avec un optimiseur Adam pour le calcul de nos gradients 
optimizer= tf.keras.optimizers.Adam(
    learning_rate = lr_schedule
)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [64]:
# Application du modèle sur 200 epochs
history = model.fit(train_data,
                    validation_data=test_data,
                    epochs=200)

Epoch 1/200


109/109 [==============================] - 91s 557ms/step - loss: 4.9370 - categorical_accuracy: 0.7898 - val_loss: 2.2094 - val_categorical_accuracy: 0.7981
Epoch 2/200
109/109 [==============================] - 46s 410ms/step - loss: 1.7831 - categorical_accuracy: 0.7991 - val_loss: 1.6303 - val_categorical_accuracy: 0.7989
Epoch 3/200
109/109 [==============================] - 47s 419ms/step - loss: 1.5349 - categorical_accuracy: 0.7976 - val_loss: 1.4476 - val_categorical_accuracy: 0.7989
Epoch 4/200
109/109 [==============================] - 46s 403ms/step - loss: 1.4156 - categorical_accuracy: 0.7969 - val_loss: 1.3594 - val_categorical_accuracy: 0.7984
Epoch 5/200
109/109 [==============================] - 48s 426ms/step - loss: 1.3321 - categorical_accuracy: 0.7987 - val_loss: 1.3039 - val_categorical_accuracy: 0.7987
Epoch 6/200
109/109 [==============================] - 48s 430ms/step - loss: 1.2951 - categorical_accuracy: 0.7980 - val_loss: 1.2616 - val_categorical_accuracy:

In [ ]:
# Test d'une traduction
for input_text, translation in test_data.take(1):
  fr_text = input_text
  pred = model.predict_classes(input_text)
  translation = translation

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [ ]:
# Phrase en français 
for indice in fr_text[0]:
  for key, value in all_fr_tokens.items():
    if indice == value:
      print(key)

Tu
as
promis
.


In [ ]:
# Phrase réelle en anglais 
for indice in np.argmax(translation, axis=-1)[0]:
  for key, value in all_en_tokens.items():
    if indice == value:
      print(key)

You
have
made
a
promise
.


In [ ]:
# Phrase traduite en anglais par le modèle 
for indice in pred[0]:
  for key, value in all_en_tokens.items():
    if indice == value:
      print(key)

You
're
to
a
a
.


In [ ]:
# Nouvel entrainement sur 500 epochs
history_2 = model.fit(train_data,
                    validation_data=test_data,
                    epochs=500)

In [ ]:
# Test sur des nouvelles traductions

for input_text, translation in test_data.take(1):
  fr_text = input_text
  pred = model.predict_classes(input_text)
  translation = translation

# French 
print("Input Sentence")
for indice in fr_text[2]:
  for key, value in all_fr_tokens.items():
    if indice == value:
      print(key)

# True
print("True Translation")
for indice in np.argmax(translation, axis=-1)[2]:
  for key, value in all_en_tokens.items():
    if indice == value:
      print(key)

# Pred
print("Model Translation")
for indice in pred[2]:
  for key, value in all_en_tokens.items():
    if indice == value:
      print(key)

Input Sentence
Vous
êtes
en
train
de
perdre
.
True Translation
You
're
losing
.
Model Translation
You
're
losing
.
